In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [1]:
!pip install --upgrade huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.0/419.0 kB 5.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [13]:
from huggingface_hub import notebook_login

In [14]:
notebook_login()

In [3]:
!pip install git+https://github.com/huggingface/transformers
!pip install huggingface_hub
!pip install accelerate
!pip install transformers[torch]
!pip install transformers
!pip install tftrainer

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ye9zecns
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ye9zecns
  Resolved https://github.com/huggingface/transformers to commit c50e0551fd5652e69360d8451a36be4f10bf04dc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 22.6 MB/s eta 0:00:00a 0:00:01
  Created wheel for transformers: filename=transformers-4.43.0.dev0-py3-none-any.whl size=9391983 sha256=091d419341d276c4293ff4e7fe0d502b2e28179fb98e48e6ca5d1f6d9fdddef7
  Stored in directory: /tmp/pip-ephem-wheel-cache-wrb681ii/wheels/04/a3/f1/b88775f8e1665827525b19ac7590250f1038d947067beba9fb
Successfully built transformers
  Attempting uninstall: safetensors
    Found e

In [5]:
!pip uninstall tokenizers
!pip install transformers

Found existing installation: tokenizers 0.19.1
Uninstalling tokenizers-0.19.1:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/tokenizers-0.19.1.dist-info/*
    /usr/local/lib/python3.11/dist-packages/tokenizers/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


In [4]:
pip install --upgrade accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 7.0 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.24.1
    Uninstalling accelerate-0.24.1:
      Successfully uninstalled accelerate-0.24.1
Note: you may need to restart the kernel to use updated packages.


In [16]:
import json
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import AutoModelForSequenceClassification
from transformers import pipeline
from transformers import Trainer
from transformers import TrainingArguments
#from tftrainer import Trainer, TrainArgument as TrainingArguments
from transformers import AutoModelForSeq2SeqLM
from torch.utils.data import Dataset
import torch
import torch.nn.functional as F
import random
import gc
import accelerate

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
gc.collect()

4704

In [23]:
class TextPairDataset(Dataset):
    def __init__(self, csv_file):
        self.df = pd.read_csv(csv_file, encoding = "utf-8")
        self.tokenizer = AutoTokenizer.from_pretrained('microsoft/Phi-3-mini-4k-instruct')
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.label_map = {'User': 0, 'AI': 1}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        question = self.df.iloc[idx]['Question']
        answer = self.df.iloc[idx]['Answer']
        label_str = self.df.iloc[idx]['User or AI']
        label = self.label_map[label_str]  

        inputs1 = self.tokenizer(question, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
        inputs2 = self.tokenizer(answer, return_tensors='pt', padding='max_length', truncation=True, max_length=128)

        input_ids = torch.cat([inputs1['input_ids'], inputs2['input_ids']], dim=1)
        attention_mask = torch.cat([inputs1['attention_mask'], inputs2['attention_mask']], dim=1)

        return {'input_ids': input_ids.flatten(), 'attention_mask': attention_mask.flatten(), 'labels': torch.tensor(label)}

In [25]:
def compute_similarity_and_reasoning(model, tokenizer, prompt, text1, text2):
    prompt_text1 = f"{prompt} {text1}"
    prompt_text2 = f"{prompt} {text2}"

    inputs1 = tokenizer(prompt_text1, return_tensors='pt', padding='max_length', truncation=True, max_length=128).to(device)
    inputs2 = tokenizer(prompt_text2, return_tensors='pt', padding='max_length', truncation=True, max_length=128).to(device)

    with torch.no_grad():
        outputs1 = model(**inputs1).logits
        outputs2 = model(**inputs2).logits

    outputs1 = outputs1 / 1e-6  
    outputs2 = outputs2 / 1e-6

    similarity_score = F.cosine_similarity(outputs1, outputs2).item()

    reasoning = "The model considers the texts to be "
    if similarity_score > 0.7:
        reasoning += "AI generated"
    elif similarity_score < 0.7 and similarity_score > 0.49:
        reasoning += "Inconclusive"
    else:
        reasoning += "Human generated."

    return similarity_score, reasoning

In [26]:
def main():
    train_dataset = TextPairDataset('/notebooks/train_dataset.csv')
    eval_dataset = TextPairDataset('/notebooks/validation_dataset.csv')

    tokenizer = AutoTokenizer.from_pretrained('microsoft/Phi-3-mini-4k-instruct')
    model =  AutoModelForCausalLM.from_pretrained('microsoft/Phi-3-mini-4k-instruct', low_cpu_mem_usage=True, num_labels=3)

    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=5000,
        per_device_train_batch_size=5000,
        per_device_eval_batch_size=5000,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        fp16=True,
        save_strategy="epoch"
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
    )

    trainer.train()

    results = trainer.evaluate()
    print(results)

    model = AutoModelForSequenceClassification.from_pretrained('./final_model').to(device)

    tokenizer.save_pretrained('./final_model_tokenizer')

In [27]:
def utilize_model(model, tokenizer, prompt_question, ai_text, user_text):
    prompt = "Compare these texts based on their relevance to the following question: " + prompt_question
    ai_text = "The cat sat."
    user_text = "A cat was sitting."
    tokenizer = AutoTokenizer.from_pretrained('./final_model')
    model = AutoModelForSequenceClassification.from_pretrained('./final_model_tokenizer')
    score, reason = compute_similarity_and_reasoning(model, tokenizer, prompt, ai_text, user_text)
    print(f"Similarity score: {score}")
    print(f"Reasoning: {reason}")

In [28]:
if __name__ == "__main__":
    main()

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)
You are not running the flash-attention implementation, expect numerical differences.


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.41 GiB. GPU 0 has a total capacty of 79.32 GiB of which 311.56 MiB is free. Process 3501710 has 79.02 GiB memory in use. Of the allocated memory 76.42 GiB is allocated by PyTorch, and 1.05 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7f492978be10>> (for post_run_cell), with arguments args (<ExecutionResult object at 7f494fa51710, execution_count=28 error_before_exec=None error_in_exec=CUDA out of memory. Tried to allocate 1.41 GiB. GPU 0 has a total capacty of 79.32 GiB of which 311.56 MiB is free. Process 3501710 has 79.02 GiB memory in use. Of the allocated memory 76.42 GiB is allocated by PyTorch, and 1.05 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF info=<ExecutionInfo object at 7f49503e4e90, raw_cell="if __name__ == "__main__":
    main()" store_history=True silent=False shell_futures=True cell_id=04200cee-f017-4114-bce8-7df92976dbc3> result=None>,),kwargs {}:


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given